Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [47]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import time

First reload the data we generated in `1_notmnist.ipynb`.

In [7]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [8]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [10]:
%%latex
$L' = L + \beta \frac{1}{2}||w||^2$

<IPython.core.display.Latex object>

In [19]:
# We add a hidden layer in the middle. ReLu(W_1 * input + b_1)

batch_size = 128
hidden_layer_node_size = 1024
beta = 0.01 # This is a good beta value to start with.

def weight_variable(shape):
  return tf.Variable(tf.truncated_normal(shape=shape))

def bias_variable(shape):
  return tf.Variable(tf.zeros(shape=shape))

def neural_network_with_one_hidden_layer(input_data):
  # Variables.
  # Training computation.
  # logits = tf.matmul(tf_train_dataset, weights) + biases 
  layer1_out = tf.nn.relu(tf.matmul(input_data, layer1_w) + layer1_b)
  layer2_out = tf.matmul(layer1_out, layer2_w) + layer2_b
  return layer2_out


graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  layer1_w = weight_variable([image_size * image_size, hidden_layer_node_size])
  print(layer1_w.get_shape())
  layer1_b = bias_variable([hidden_layer_node_size])
  print(layer1_b.get_shape())
  layer2_w = weight_variable([hidden_layer_node_size, num_labels])
  print(layer2_w.get_shape())
  layer2_b = bias_variable([num_labels])
  print(layer2_b.get_shape())
  
  train_model = neural_network_with_one_hidden_layer(tf_train_dataset)
  valid_model = neural_network_with_one_hidden_layer(tf_valid_dataset)  
  test_model = neural_network_with_one_hidden_layer(tf_test_dataset)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_model))

  # Add L2 Regularization for each weight
  loss += beta * tf.nn.l2_loss(layer1_w) + beta * tf.nn.l2_loss(layer2_w) 
  
  # Optimizer.
  gdo = tf.train.GradientDescentOptimizer
  optimizer = gdo(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_model)
  valid_prediction = tf.nn.softmax(valid_model)
  test_prediction = tf.nn.softmax(test_model)

(784, 1024)
(1024,)
(1024, 10)
(10,)


In [20]:
num_steps = 4301 # Tuned to be the highest around 3001 to 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3568.416260
Minibatch accuracy: 5.5%
Validation accuracy: 14.1%
Minibatch loss at step 500: 1913.866943
Minibatch accuracy: 78.9%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 1155.199219
Minibatch accuracy: 78.1%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 698.003723
Minibatch accuracy: 79.7%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 421.362579
Minibatch accuracy: 85.9%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 255.532364
Minibatch accuracy: 84.4%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 154.725296
Minibatch accuracy: 88.3%
Validation accuracy: 84.6%
Minibatch loss at step 3500: 93.877762
Minibatch accuracy: 86.7%
Validation accuracy: 85.4%
Minibatch loss at step 4000: 57.180344
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Test accuracy: 92.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
# We add a hidden layer in the middle. ReLu(W_1 * input + b_1)

batch_size = 128
hidden_layer_node_size = 1024
beta = 0.01 # This is a good beta value to start with.
num_steps = 3001
reduce_the_dataset_range = 1000

def weight_variable(shape):
  return tf.Variable(tf.truncated_normal(shape=shape))

def bias_variable(shape):
  return tf.Variable(tf.zeros(shape=shape))

def neural_network_with_one_hidden_layer(input_data):
  # Variables.
  # Training computation.
  # logits = tf.matmul(tf_train_dataset, weights) + biases 
  layer1_out = tf.nn.relu(tf.matmul(input_data, layer1_w) + layer1_b)
  layer2_out = tf.matmul(layer1_out, layer2_w) + layer2_b
  return layer2_out


graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  layer1_w = weight_variable([image_size * image_size, hidden_layer_node_size])
  print(layer1_w.get_shape())
  layer1_b = bias_variable([hidden_layer_node_size])
  print(layer1_b.get_shape())
  layer2_w = weight_variable([hidden_layer_node_size, num_labels])
  print(layer2_w.get_shape())
  layer2_b = bias_variable([num_labels])
  print(layer2_b.get_shape())
  
  train_model = neural_network_with_one_hidden_layer(tf_train_dataset)
  valid_model = neural_network_with_one_hidden_layer(tf_valid_dataset)  
  test_model = neural_network_with_one_hidden_layer(tf_test_dataset)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_model))

  # Add L2 Regularization for each weight
  loss += beta * tf.nn.l2_loss(layer1_w) + beta * tf.nn.l2_loss(layer2_w) 
  
  # Optimizer.
  gdo = tf.train.GradientDescentOptimizer
  optimizer = gdo(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_model)
  valid_prediction = tf.nn.softmax(valid_model)
  test_prediction = tf.nn.softmax(test_model)
    

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] / reduce_the_dataset_range - batch_size)
    # Generate a minibatch.
    print("Train using dataset range: " + str(offset) + "-" + str(offset + batch_size))
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides [nn.dropout()](https://www.tensorflow.org/api_docs/python/tf/nn/dropout) for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Normal case, large train_dataset

In [57]:
# We add a hidden layer in the middle. ReLu(W_1 * input + b_1)

learning_rate = 0.01
batch_size = 128
hidden_layer_node_size = 1024
beta = 0.01 # This is a good beta value to start with.
num_steps = 15001
dropout = 0.75 # Dropout, probability to keep units

def weight_variable(shape):
  return tf.Variable(tf.truncated_normal(shape=shape))

def bias_variable(shape):
  return tf.Variable(tf.zeros(shape=shape))

def neural_network_with_one_hidden_layer_with_dropout(input_data, drop_out):
  # Variables.
  # Training computation.
  # logits = tf.matmul(tf_train_dataset, weights) + biases 
  layer1_out = tf.nn.dropout(tf.nn.relu(tf.matmul(input_data, layer1_w) + layer1_b), drop_out)
  layer2_out = tf.matmul(layer1_out, layer2_w) + layer2_b
  return layer2_out


graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  # dropout (keep probability), for training, we put 0.75, for validation/test, we put 1.0 for no-dropout
  keep_prob = tf.placeholder(tf.float32)
    
  layer1_w = weight_variable([image_size * image_size, hidden_layer_node_size])
  print(layer1_w.get_shape())
  layer1_b = bias_variable([hidden_layer_node_size])
  print(layer1_b.get_shape())
  layer2_w = weight_variable([hidden_layer_node_size, num_labels])
  print(layer2_w.get_shape())
  layer2_b = bias_variable([num_labels])
  print(layer2_b.get_shape())    
  
  train_model = neural_network_with_one_hidden_layer_with_dropout(tf_train_dataset, keep_prob)
  valid_model = neural_network_with_one_hidden_layer_with_dropout(tf_valid_dataset, keep_prob)  
  test_model = neural_network_with_one_hidden_layer_with_dropout(tf_test_dataset, keep_prob)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_model))

  # Add L2 Regularization for each weight
  loss += beta * tf.nn.l2_loss(layer1_w) + beta * tf.nn.l2_loss(layer2_w) 
  
  # Optimizer.
  gdo = tf.train.GradientDescentOptimizer
  optimizer = gdo(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_model)
  valid_prediction = tf.nn.softmax(valid_model)
  test_prediction = tf.nn.softmax(test_model)
    

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
#     print("Train using dataset range: " + str(offset) + "-" + str(offset + batch_size))
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {
        tf_train_dataset: batch_data, 
        tf_train_labels: batch_labels,
        keep_prob: dropout
    }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))        

(784, 1024)
(1024,)
(1024, 10)
(10,)
Initialized
Minibatch loss at step 0: 3559.415527
Minibatch accuracy: 5.5%
Validation accuracy: 10.6%
Minibatch loss at step 500: 2896.900879
Minibatch accuracy: 65.6%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 2599.684814
Minibatch accuracy: 73.4%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 2339.171631
Minibatch accuracy: 81.2%
Validation accuracy: 82.1%
Minibatch loss at step 2000: 2122.452637
Minibatch accuracy: 78.9%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 1910.315063
Minibatch accuracy: 76.6%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 1722.477173
Minibatch accuracy: 76.6%
Validation accuracy: 83.0%
Minibatch loss at step 3500: 1554.579712
Minibatch accuracy: 82.0%
Validation accuracy: 83.4%
Minibatch loss at step 4000: 1410.526123
Minibatch accuracy: 79.7%
Validation accuracy: 83.5%
Minibatch loss at step 4500: 1271.957520
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Minibatch loss a

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
# https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter

from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [62]:
start_learning_rate = 0.1
decay_steps = 10000
decay_rate = 0.85
batch_size = 128
enable_l2_reg = True
beta = 0.01 # This is a good beta value to start with.
num_steps = 160001
dropout = 0.5 # Dropout, probability to keep units
hidden_layer_count = 5
first_layer_nodes = 1024
all_layer_nodes = [image_size * image_size]
all_layer_nodes.extend([first_layer_nodes / pow(2, n) for n in range(hidden_layer_count)])
all_layer_nodes.append(num_labels)    

def weight_variable(shape):
  return tf.Variable(tf.truncated_normal(shape=shape, stddev=math.sqrt(2.0 / shape[0])))

def bias_variable(shape):
  return tf.Variable(tf.zeros(shape=shape))

weights = [
  weight_variable([all_layer_nodes[i], all_layer_nodes[i + 1]])
  for i in range(len(all_layer_nodes) - 1)
]

biases = [
  bias_variable([all_layer_nodes[i]]) for i in range(1, len(all_layer_nodes))
]

# dropout (keep probability), for training, we put 0.75, for validation/test, we put 1.0 for no-dropout
keep_prob = tf.placeholder(tf.float32)

for i in range(len(weights)):
    print(weights[i].shape) 
for i in range(len(biases)):
    print(biases[i].shape) 

(784, 1024)
(1024, 512)
(512, 256)
(256, 128)
(128, 64)
(64, 10)
(1024,)
(512,)
(256,)
(128,)
(64,)
(10,)


In [67]:
def multi_layer_model(input_data):
  each_layer = input_data
  for i in range(len(all_layer_nodes) - 2):
    each_layer = tf.nn.dropout(
        tf.nn.relu(tf.matmul(each_layer, weights[i]) + biases[i]), 
        keep_prob)
  # Add output layer
  return tf.add(tf.matmul(each_layer, weights[-1]), biases[-1])

# graph = tf.Graph()
# with graph.as_default():

# Input data. For the training data, we use a placeholder that will be fed
# at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)
  
train_model = multi_layer_model(tf_train_dataset)
valid_model = multi_layer_model(tf_valid_dataset)  
test_model = multi_layer_model(tf_test_dataset)
    
if enable_l2_reg:
    unreg_loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_model)
    l2_loss = beta * tf.nn.l2_loss(weights[0])
    for i in range(1, len(weights)):
        l2_loss += beta * tf.nn.l2_loss(weights[i])
    loss = unreg_loss + l2_loss
else:
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_model))
# loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_model)
# # Add L2 Regularization for each weight
# loss += 
# regularization = tf.reduce_sum([beta * tf.nn.l2_loss(weights[i]) 
#                                 for i in range(len(weights))])
# loss = tf.reduce_mean([loss, regularization])
  
# Optimizer.
# gdo = tf.train.GradientDescentOptimizer
# optimizer = gdo(start_learning_rate).minimize(loss)

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(
    start_learning_rate, global_step, decay_steps, decay_rate, staircase=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)\
                    .minimize(loss, global_step=global_step)
  
# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(train_model)
valid_prediction = tf.nn.softmax(valid_model)
test_prediction = tf.nn.softmax(test_model)
    
# show_graph(tf.get_default_graph().as_graph_def())    
    
# Initializing the variables
init = tf.global_variables_initializer()    
    
    
with tf.Session() as session:
  print("Initialized")  
  session.run(init)
  start_time = time.time()
    
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
#     print("Train using dataset range: " + str(offset) + "-" + str(offset + batch_size))
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {
        tf_train_dataset: batch_data, 
        tf_train_labels: batch_labels,
        keep_prob: dropout
    }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Time Spent So Far: %d sec" % (time.time() - start_time))
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
        
  print("Total Time Spent: %d sec" % (time.time() - start_time))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))  

Initialized
Time Spent So Far: 0 sec


TypeError: float argument required, not numpy.ndarray